In [3]:
# https://www.johndcook.com/blog/2021/04/20/magic-square-of-squares/

# create n-by-n square, with
n = 4

# largest entry (before squaring)
N = 260

# squares that could be used
squares = [ i**2 for i in range(1,N+1) ]

print("n =",n)
print("N =",N)
print("squares =",squares)
print(squares[N-1])

n = 4
N = 260
squares = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801, 10000, 10201, 10404, 10609, 10816, 11025, 11236, 11449, 11664, 11881, 12100, 12321, 12544, 12769, 12996, 13225, 13456, 13689, 13924, 14161, 14400, 14641, 14884, 15129, 15376, 15625, 15876, 16129, 16384, 16641, 16900, 17161, 17424, 17689, 17956, 18225, 18496, 18769, 19044, 19321, 19600, 19881, 20164, 20449, 20736, 21025, 21316, 21609, 21904, 22201, 22500, 22801, 23104, 23409, 23716, 24025, 24336, 24649, 24964, 25281, 2560

In [4]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# x[i,j] is the entry in the magic square in row i, column j
x = m.addVars(n,n,vtype=GRB.CONTINUOUS)

# y[i,j,k] = 1 if row i column j has squares[k] in it
y = m.addVars(n,n,N,vtype=GRB.BINARY)

# What is the row/column/diagonal sum?
z = m.addVar(vtype=GRB.CONTINUOUS)

In [5]:
# Each cell has one square in it
m.addConstrs( gp.quicksum( y[i,j,k] for k in range(N) ) == 1 for i in range(n) for j in range(n) )

# Relate x and y
m.addConstrs( gp.quicksum( squares[k]*y[i,j,k] for k in range(N) ) == x[i,j] for i in range(n) for j in range(n) )

# Each row should sum to z
m.addConstrs( gp.quicksum( x[i,j] for j in range(n)) == z for i in range(n) )

# Each column should sum to z
m.addConstrs( gp.quicksum( x[i,j] for i in range(n)) == z for j in range(n) )

# The main diagonals should sum to z
m.addConstr( gp.quicksum( x[i,i] for i in range(n) ) == z )
m.addConstr( gp.quicksum( x[i,n-i-1] for i in range(n) ) == z )

# Each square should be used at most once as an entry
m.addConstrs( gp.quicksum( y[i,j,k] for i in range(n) for j in range(n) ) <= 1 for k in range(N) )

m.update()

In [ ]:
# minimize the row/column/diagonal sums (equivalently, the sum of entries)
m.setObjective( z, GRB.MINIMIZE)

# Inject a warm start solution
if N >= 260:
    AWJ = [ 
        [30**2, 246**2, 172**2, 45**2], 
        [93**2, 116**2, 66**2, 258**2], 
        [126**2, 138**2, 237**2, 44**2], 
        [260**2, 3**2, 54**2, 150**2], 
    ]

    for i in range(n):
        for j in range(n):
            x[i,j].Start = AWJ[i][j]
        
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 302 rows, 4177 columns and 12546 nonzeros
Model fingerprint: 0x1295b3a9
Variable types: 17 continuous, 4160 integer (4160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 93025 (0.01s)
Loaded user MIP start with objective 93025

Presolve time: 0.04s
Presolved: 302 rows, 4177 columns, 12530 nonzeros
Variable types: 0 continuous, 4177 integer (4160 binary)

Root relaxation: objective 3.740000e+02, 210 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  374.00000    0   25 93025.0000  374.00000   100%     -    0s
     0     0  374.00000 

In [ ]:
sol = [ [ x[i,j].x for i in range(n) ] for j in range(n) ]
print(sol)